## Collectting bananas using Reinforced Q-learning
Using action values for is a classic way of solving reinforced learning problems (RL). For small problems with limited state spaces and actions tabular methods are used. Temporal methods using Q-learning is a good way to solve these problems where the “world” we are navigating is a known place. The Bellman equation shows the update rule

$$\begin{gather*}
  Q^*(s, \, a) = \mathbf{E}_{s'} \left [r + \gamma \underset{a'}{\max} \, Q^{*} (s', a')|\,  s ,\, a\right ] \;
\end{gather*}$$

Most real problems turn out to be much more complex and needs other methods. A novel approach is the Deep Q-learning (ref) which can be used to solve much bigger problems with e.g. bitmaps as state space. To be able to compute the state action values $Q(s,a)$ in larger state spaces they propose to use estimators (of function approximators) using neural networks to estimate the Q values. $Q^*$ is the optimal values. 

$$Q(s,a; \theta) ≈ Q^* (s, a)$$

The update rule for the weights in deep Q-learning looks like this.

$$\begin {gather*}
\Delta \theta = \alpha \cdot \overbrace{( \underbrace{R + \gamma \max_a Q(S', a, \theta^-)}_{\rm {TD~target}} - \underbrace{Q(S, A, \theta)}_{\rm {old~value}})}^{\rm {TD~error}} \nabla_\theta Q(S, A, \theta) \;
\end{gather*}$$

### Implementation

### Neural network
I have implemented my solution based on the Q-learning algorithm in the course. The neural networks (two identical) have two layer networks with one input layer, one hidden layer.


The size of the input state is 37 and the input and hidden layer have 256 units. RELU activation is used.


### Learning using DQN

The networks are trained using an error function and backpropagation. There are problems with using the neural networks directly for this since they turn out to be quite unstable since they are non-linear and that they learn correlations between the state action pairs in the training phase. 

The DQN method introduced two changes to mend this problem. By using a replay buffer (of s,a pairs) the training of the network is done by randomly picking a subset of experienced s,a pairs and not the actual experienced pairs of the current episode. $e_t = (s_t, a_t, r_t, s_{t+1})$ 

The second DQN idea is to introduce two networks such that we can update the target network only periodically to reduce the correlations with the target. The separate target network with parameters $\theta^{-}$ for generating the targets in the Q-learning update, and it´s cloned every $C$ updates the parameters of the network $Q(s, a; \;\theta)$ to obtain the target network $Q(s, a; \;\theta^{-})$.

The cloning is done with a soft update i.e. making a (close) copy using the parameter $\tau$ using the update equation $\theta^- = \tau * \theta + (1 - \tau)*\theta^-$

The parameters I used in the solution is shown below. The solution (better than 13 over 100 epsiodes) was found in 659 episodes.


In [ ]:
BUFFER_SIZE = int(1e5)  # replay buffer size 1e5
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the target network

### DQN rewards plot 
<img src="dqn_learning.png">

### Learning using Double DQN (DDQN)

DQN uses the same local network (paramters named $\theta$)to select and evaluate an action.

\begin{gather*}
  Y_t = r + \gamma Q (s', \underset{a'}{\arg \max} \, Q (s', a'; \theta);\theta) \;
\end{gather*}

where $Y_t$ corresponds to the target value. In double Q-learning (DDQN) we use both networks, $\theta$ to select the best action and $\theta^-$ to evaluate the action like this:

\begin{gather*}
  Y_t = r + \gamma Q (s', \underset{a'}{\arg \max} \, Q (s', a'; \theta);\theta^{-}) \;
\end{gather*}

Both networks must agree over the best action to the Q-value returned to be the maximum one. Learning the banana environment with this algorithm found a solution faster the DQN at 447 epsisodes.



### DDQN rewards plot
<img src="ddqn_learning.png">

## Ideas for future work
Since the first publication of the DQN there have been several improvements to the algorithm that can be investigated and implemented. I only implemented the "easy" DDQN above, but there are other improvements like:

### Duelling DQN
### Prioritized Experience Replay
In this improvement we also keep the error each sample has. The idea is that the samples with the biggest errors are the ones we can learn the most from. Introducing this concept will need two more parameters to control the stochastic sampling (non-uniform) and avoid stopping exploring the state space.